In [1]:
! pip install mne==0.19.2
! pip install torch
! pip install skorch

     |████████████████████████████████| 6.4MB 11.4MB/s 
     |████████████████████████████████| 133kB 14.0MB/s 


In [2]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/TSception/result_gs_sliceDataset/'
import os
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

/content/drive/MyDrive


In [3]:
import sys, importlib
#importlib.reload(sys.modules['grasp.config'])
from grasp.config import *
# orveride the data_dir in config file
#data_dir='/content/drive/MyDrive/data/' # googleDrive

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json


In [4]:
data_dir

'/content/drive/MyDrive/data/'

In [5]:
import torch
from skorch.helper import predefined_split, SliceDataset
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from skorch.callbacks import Callback
import numpy as np
import matplotlib.pyplot as plt

from grasp.TSception.Models import TSception2
from grasp.utils import raw_input,freq_input,SEEGDataset,set_random_seeds,cuda_or_cup
from grasp.config import root_dir, tmp_dir
from grasp.myskorch import MyRegressor, on_epoch_begin_callback, on_batch_end_callback, on_epoch_end_callback


In [6]:
# reload function
import sys, importlib
importlib.reload(sys.modules['grasp.TSception.Models'])
importlib.reload(sys.modules['grasp.utils'])
importlib.reload(sys.modules['grasp.config'])
from grasp.TSception.Models import TSception,TSception2
from grasp.utils import freq_input, SEEGDataset, set_random_seeds, cuda_or_cup, windTo3D_x, windTo3D_y, SEEGDataset3D, \
    raw_input

In [7]:
# import from another folder
import sys
sys.path.insert(1, '/content/drive/MyDrive/examples')
from IMV_LSTM.networks import IMVTensorLSTM

In [8]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(rawData2));

In [9]:
device=cuda_or_cup()
seed = 123456789  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed)


GPU computing:   True


In [10]:
%%capture
# suppress the output
sid=6
sampling_rate=1000
traindata, valdata, testdata = raw_input(sid,split=True,move2=True)  # (chns, 15000/15001, 118) (channels, time, trials)
##traindata, valdata, testdata = rawData2('raw','all',move2=True)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)

trainx, trainy = traindata[:, :-2, :], traindata[:, -2, :] #-2 is real force, -1 is target
valx, valy = valdata[:, :-2, :], valdata[:, -2, :]
testx, testy = testdata[:, :-2, :], testdata[:, -2, :]
# trainx.shape(trials, channels,times)
# trainy.shape(trials, times)

In [11]:
# put train and valdate togather, CV will do the split automatically
train_and_valx=np.concatenate((trainx,valx),axis=0)
train_and_valy=np.concatenate((trainy,valy),axis=0)

In [12]:
#samples=trainx.shape[0]
chnNum=train_and_valx.shape[1]
step=50 #ms
T=1000 #ms

train_and_val_ds = SEEGDataset3D(train_and_valx, train_and_valy,T, step)
train_ds = SEEGDataset3D(trainx, trainy,T, step)
val_ds = SEEGDataset3D(valx, valy,T, step)
test_ds = SEEGDataset3D(testx, testy,T, step)

train_and_valy=windTo3D_y(train_and_valy,T,step).astype(np.float32)
#Note: no dataset when gridsearch:  https://skorch.readthedocs.io/en/stable/user/FAQ.html#how-do-i-use-sklearn-gridseachcv-when-my-data-is-in-a-dataset
X_sl = SliceDataset(train_and_val_ds, idx=0)  # idx=0 is the default
y_sl = SliceDataset(train_and_val_ds, idx=1)


In [13]:
learning_rate=0.001
num_T = 3 # (6 conv2d layers) * ( 3 kernel each layer)
num_S = 3
num_ST=3
dropout=0.5
#Lambda = 1e-6
Lambda = 0
totalLen=train_and_valx.shape[2] #ms
batch_size=int((totalLen-T)/step) # 280
# do not initialize here. Do it in net definition.
#model=TSception2(T, step, sampling_rate,chnNum, num_T, num_S,batch_size,dropout).float()

#if(enable_cuda):
#	model.cuda()

In [14]:
from skorch.callbacks import Checkpoint, TrainEndCheckpoint, EarlyStopping
cp = Checkpoint(dirname=result_dir) # Will automatically save the model when there is an improvement during validation.
train_end_cp = TrainEndCheckpoint(dirname=result_dir) # save model and the end of the training

early_stopping_callbacks = EarlyStopping(monitor='valid_loss', lower_is_better=True, patience=5)

In [15]:
net = MyRegressor(
    module=TSception2,
    module__chnNum=chnNum,
    module__sampling_rate=sampling_rate,
    module__num_T=num_ST,
    module__num_S=num_ST,
    module__dropout=dropout,
    lambda1=Lambda, # can be at at any sequence location.
    iterator_train__shuffle=True,

    #train_split=predefined_split(val_ds), #disable this when do grid search
    train_split=None, # set to be None when doing grid search
    verbose=0, # grid search

    max_epochs=50,
    lr=learning_rate,
    batch_size=1,
    optimizer=torch.optim.Adam,
    criterion = nn.MSELoss,
    callbacks=[('on_epoch_begin_callback', on_epoch_begin_callback),('on_batch_end_callback',on_batch_end_callback),
               ],# ('on_epoch_end_callback', on_epoch_end_callback(result_dir)),
               #cp,train_end_cp], # early_stopping_callbacks
    device = device
)

In [16]:
params = {
    'lr': [0.9, 0.003],
    #'module__num_S':[2,3],
    'lambda1':[1e-6,1],
    'module__num_T':[1,3],
    #'module__dropout':[0.1,0.3,0.5,0.7],
    #'optimizer': [torch.optim.Adam, torch.optim.Adagrad ,torch.optim.SGD],
    #'optimizer__weight_decay':[0, 1e-04, 1e-03],
    'max_epochs': [1,100],
    #'module__num_units': [10, 20],
}
'''
params = {
    'lr': [0.001, 0.003,0.005,0.01],
    'module__num_S':[2,3,4,5,6],
    'module__num_T':[2,3,4,5,6],
    'module__dropout':[0.1,0.3,0.5,0.7],
    'optimizer': [torch.optim.Adam, torch.optim.Adagrad ,torch.optim.SGD],
    'optimizer__weight_decay':[0, 1e-04, 1e-03],
    #'max_epochs': [100,200, 300],
    #'module__num_units': [10, 20],
}
'''
# note
# 1, gridsearch will do the training and validate set split itself. so set train_split=False according to https://nbviewer.jupyter.org/github/skorch-dev/skorch/blob/master/notebooks/Basic_Usage.ipynb#Performing-a-grid-search
# 2, how to search on different optimizor?

"\nparams = {\n    'lr': [0.001, 0.003,0.005,0.01],\n    'module__num_S':[2,3,4,5,6],\n    'module__num_T':[2,3,4,5,6],\n    'module__dropout':[0.1,0.3,0.5,0.7],\n    'optimizer': [torch.optim.Adam, torch.optim.Adagrad ,torch.optim.SGD],\n    'optimizer__weight_decay':[0, 1e-04, 1e-03],\n    #'max_epochs': [100,200, 300],\n    #'module__num_units': [10, 20],\n}\n"

In [ ]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(net, params, refit=False, cv=5, scoring='neg_mean_squared_error', verbose=2) # score: higher is better
gs.fit(X_sl,train_and_valy)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1 ............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


On Epoch begin 1.


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
RuntimeError: Given groups=1, weight of size [18, 18, 19, 1], expected input[280, 6, 19, 124] to have 18 channels, but got 6 channels instead

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.6s remaining:    0.0s


[CV]  lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1, total=   0.3s
[CV] lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1 ............
On Epoch begin 1.
[CV]  lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1, total=   0.0s
[CV] lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1 ............
On Epoch begin 1.
[CV]  lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1, total=   0.0s
[CV] lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1 ............
On Epoch begin 1.
[CV]  lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1, total=   0.0s
[CV] lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1 ............
On Epoch begin 1.
[CV]  lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=1, total=   0.0s
[CV] lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=3 ............
On Epoch begin 1.
[CV]  lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=3, total=   9.6s
[CV] lambda1=1e-06, lr=0.9, max_epochs=1, module__num_T=3 ............
On Epoch begin 1.
[CV]  lambda1=1e-06, l

...

[CV]  lambda1=1, lr=0.003, max_epochs=1, module__num_T=1, total=   0.0s
[CV] lambda1=1, lr=0.003, max_epochs=1, module__num_T=3 ..............
On Epoch begin 1.
[CV]  lambda1=1, lr=0.003, max_epochs=1, module__num_T=3, total=   9.0s
[CV] lambda1=1, lr=0.003, max_epochs=1, module__num_T=3 ..............
On Epoch begin 1.
[CV]  lambda1=1, lr=0.003, max_epochs=1, module__num_T=3, total=   9.1s
[CV] lambda1=1, lr=0.003, max_epochs=1, module__num_T=3 ..............
On Epoch begin 1.
[CV]  lambda1=1, lr=0.003, max_epochs=1, module__num_T=3, total=   9.0s
[CV] lambda1=1, lr=0.003, max_epochs=1, module__num_T=3 ..............
On Epoch begin 1.
[CV]  lambda1=1, lr=0.003, max_epochs=1, module__num_T=3, total=   9.1s
[CV] lambda1=1, lr=0.003, max_epochs=1, module__num_T=3 ..............
On Epoch begin 1.
[CV]  lambda1=1, lr=0.003, max_epochs=1, module__num_T=3, total=   9.0s
[CV] lambda1=1, lr=0.003, max_epochs=100, module__num_T=1 ............
On Epoch begin 1.
[CV]  lambda1=1, lr=0.003, max_epo

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
RuntimeError: Given groups=1, weight of size [18, 18, 19, 1], expected input[280, 6, 19, 124] to have 18 channels, but got 6 channels instead

  FitFailedWarning)


[CV]  lambda1=1, lr=0.003, max_epochs=100, module__num_T=1, total=   0.0s
[CV] lambda1=1, lr=0.003, max_epochs=100, module__num_T=3 ............
On Epoch begin 1.
On Epoch begin 2.
On Epoch begin 3.
On Epoch begin 4.
On Epoch begin 5.
On Epoch begin 6.
On Epoch begin 7.
On Epoch begin 8.
On Epoch begin 9.
On Epoch begin 10.
On Epoch begin 11.
On Epoch begin 12.
On Epoch begin 13.
On Epoch begin 14.
On Epoch begin 15.
On Epoch begin 16.
On Epoch begin 17.
On Epoch begin 18.
On Epoch begin 19.
On Epoch begin 20.
On Epoch begin 21.
On Epoch begin 22.
On Epoch begin 23.
On Epoch begin 24.
On Epoch begin 25.
On Epoch begin 26.
On Epoch begin 27.
On Epoch begin 28.
On Epoch begin 29.
On Epoch begin 30.
On Epoch begin 31.
On Epoch begin 32.
On Epoch begin 33.
On Epoch begin 34.
On Epoch begin 35.
On Epoch begin 36.
On Epoch begin 37.
On Epoch begin 38.
On Epoch begin 39.
On Epoch begin 40.
On Epoch begin 41.
On Epoch begin 42.
On Epoch begin 43.
On Epoch begin 44.
On Epoch begin 45.
On Epoch 

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 267.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=<class 'grasp.myskorch.MyRegressor'>[uninitialized](
  module=<class 'grasp.TSception.Models.TSception2'>,
  module__chnNum=19,
  module__dropout=0.5,
  module__num_S=3,
  module__num_T=3,
  module__sampling_rate=1000,
),
             iid='deprecated', n_jobs=None,
             param_grid={'lambda1': [1e-06, 1], 'lr': [0.9, 0.003],
                         'max_epochs': [1, 100], 'module__num_T': [1, 3]},
             pre_dispatch='2*n_jobs', refit=False, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [ ]:
print(gs.best_score_, gs.best_params_) # -0.21898927688598632 {'lr': 0.001}

-0.21868969798088073 {'lambda1': 1e-06, 'lr': 0.003, 'max_epochs': 100, 'module__num_T': 3}


In [ ]:
#lr_fitted=gs.best_params_['lr']

In [ ]:

net2 = MyRegressor(
    module=TSception2,
    module__chnNum=chnNum,
    module__sampling_rate=sampling_rate,
    module__num_T=num_ST,
    module__num_S=num_ST,
    module__dropout=dropout,
    lambda1=Lambda, # can be at at any sequence location.
    iterator_train__shuffle=True,

    train_split=predefined_split(val_ds), #disable this when do grid search
    #train_split=None, # set to be None when doing grid search
    #verbose=0, # set to 0 when grid search

    max_epochs=200,
    lr=0.001,
    batch_size=1,
    optimizer=torch.optim.Adam,
    criterion = nn.MSELoss,
    callbacks=[('on_epoch_begin_callback', on_epoch_begin_callback),('on_batch_end_callback',on_batch_end_callback),
               ('on_epoch_end_callback', on_epoch_end_callback(result_dir)),],
               #cp,train_end_cp], # early_stopping_callbacks
    device = device
)

In [ ]:
net2.fit(train_ds,y=None)